# I. Import Needed Libraries

Selenium: accesses and controls web browser to submit search and locate web elements by xpath

CSV: reads and writes CSV files

re: regular expression for pattern matching

requests: makes HTTP requests

BeautifulSoup:parses HTML

pandas: data analysis library to manage dataframes


In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv, re, requests
from bs4 import BeautifulSoup
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


# II. Instantiate Browser Object 

In [2]:
url = 'https://ww2.amstat.org/meetings/jsm/2016/onlineprogram/'

# chromedriver needs to be in exectuable path or you need to map to where your chromedriver is
browser = webdriver.Chrome()
browser.get(url)


# III. Navigate through webpage and scrape objects

### Creat xpaths for needed items

In [53]:
xpath_search = '//*[@id="ui-accordion-accordion-panel-0"]/table/tbody/tr/td/form/input[3]'
xpath_session = '//*[@id="bottom"]/table/tbody/tr[1]/td/table/tbody/tr[1]/td[1]/a[contains(@href, "SessionID")]'
# '//tbody//tr[1]//td[1]//a[contains(@href, "SessionID")]'
xpath_titles = '//tbody//tr[2]//td//b'
xpath_sponsors = '//tbody//tr[3]//td'
xpath_type = '//tbody//tr/td[4]'

### Click search with empty fields

In [4]:
# wouldn't click on this page so had to execute script instead of standard click() method
element = browser.find_element_by_xpath(xpath_search)
browser.execute_script("arguments[0].click();", element)


# browser.find_element_by_xpath(xpath_search).click()
# wait = WebDriverWait(browser, 30)
# wait.until(EC.presence_of_element_located((By.XPATH, xpath_search)))
# browser.find_element_by_xpath(xpath_search).click()

### Create lists of each field needed

In [56]:
sessions = [item.text for item in browser.find_elements_by_xpath(xpath_session) if 'Session' not in item.text]
titles = [item.text for item in browser.find_elements_by_xpath(xpath_titles)]
sponsors = [item.text.replace('\n', ', ') for item in browser.find_elements_by_xpath(xpath_sponsors) 
            if item.text != 'Sponsor(s)']

session_linkToAbstract = [item.get_attribute('href') for item in browser.find_elements_by_xpath(xpath_session)]

In [61]:
session_type = [item.split(' — ')[1] for item in titles]
titles = [item.split(' — ')[0] for item in titles]

In [64]:
len(sessions) == len(titles) == len(sponsors) == len(session_linkToAbstract) == len(session_type)

True

In [65]:
df = pd.DataFrame({'Session_No' : sessions,
                   'Session_Title' : titles,
                   'Sponsors': sponsors,
                   'Type': session_type,
                   'Session_URL': session_linkToAbstract                   
                  })

In [66]:
df['Session_ID'] = df['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))

In [67]:
df.head(1)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
0,213714,Committee on Applied Statisticians Meeting,https://ww2.amstat.org/meetings/jsm/2016/onlin...,Committee on Applied Statisticians,Other Cmte/Business,213714


In [68]:
df.drop_duplicates(subset = 'Session_URL', inplace = True)
df.reset_index(inplace = True, drop= True)

In [69]:
dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Datasets'
df.to_excel(r'{}\2016Sessions.xlsx'.format(dest))

# Test Code for this year's object locations (xpaths)

In [70]:
df[df['Session_No'].apply(lambda x: bool(re.search(r'(?<!\d)\d{1,3}(?!\d)',x)))]

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
8,CE_02C,Best Practices in Data Visualization: Present ...,https://ww2.amstat.org/meetings/jsm/2016/onlin...,ASA,Professional Development Continuing Education ...,212647
9,CE_43P,"Effective Collaboration, Part I (ADDED FEE)",https://ww2.amstat.org/meetings/jsm/2016/onlin...,ASA,Professional Development Personal Skills Devel...,213251
11,CE_01C,"Introduction to Bayesian Methods, Computation,...",https://ww2.amstat.org/meetings/jsm/2016/onlin...,ASA,Professional Development Continuing Education ...,213231
12,CE_03C,Applied Longitudinal Analysis (ADDED FEE),https://ww2.amstat.org/meetings/jsm/2016/onlin...,"ASA , Biometrics Section",Professional Development Continuing Education ...,212630
13,CE_04C,A Primer to Web Scraping with R (ADDED FEE),https://ww2.amstat.org/meetings/jsm/2016/onlin...,ASA,Professional Development Continuing Education ...,213233
14,CE_05C,Advanced Topics in Survey Sampling (ADDED FEE),https://ww2.amstat.org/meetings/jsm/2016/onlin...,"ASA , Survey Research Methods Section",Professional Development Continuing Education ...,212433
15,CE_06C,Statistical Analysis of Financial Data with R ...,https://ww2.amstat.org/meetings/jsm/2016/onlin...,"ASA , Business and Economic Statistics Section",Professional Development Continuing Education ...,212653
18,CE_07C,Introduction to Bayesian Inference with Stan a...,https://ww2.amstat.org/meetings/jsm/2016/onlin...,ASA,Professional Development Continuing Education ...,213234
19,CE_44P,Preparing Statisticians for Leadership: How to...,https://ww2.amstat.org/meetings/jsm/2016/onlin...,ASA,Professional Development Personal Skills Devel...,213253
32,CE_08C,An Example-Driven Hands-On Introduction to Rcp...,https://ww2.amstat.org/meetings/jsm/2016/onlin...,ASA,Professional Development Continuing Education ...,213235


In [95]:
test = df['Session_URL'].iloc[8]
test

'https://ww2.amstat.org/meetings/jsm/2016/onlineprogram/ActivityDetails.cfm?SessionID=212647'

In [96]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [98]:
re.sub(r'(\n|\r|\t)', '', soup.find_all('tr')[5].text)

"Do you get questions about your data after you think you presented them clearly? Do people's eyes glaze over when you show them your results? In this presentation, learn how to apply simple guidelines to improve how you display your statistical data (results) visually. As with verbal communication, there exists a grammar and vocabulary of visual communication that will help you get your point across clearly, accurately, and even beautifully. This course is designed for anyone with a desire to communicate data clearly in a visual form. No prerequisite is required."

# Run Code to extract workshops and abstract_urls

In [81]:
# dataframe of abstracts to scrape with new urls
session_url = []
abstract_url = []
abstract_title = []
session_no_v2 = []

#dataframe of abstracts already written in the session
workshop_abstract = []
workshop_session_no = []
workshop_session_url = []

for link in session_linkToAbstract:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # extract abstract titles and links
    abstract_links = [item.get('href') for item in soup.find_all('a') if re.search('AbstractDetails', str(item.get('href')))]
    
    if len(abstract_links) == 0:
        try:
            abstract_workshop = re.sub(r'(\n|\r|\t)', '', soup.find_all('tr')[5].text)
            id_no = soup.find_all('tr')[2].find('b').text.strip()
            
            workshop_abstract.append(abstract_workshop)
            workshop_session_no.append(id_no)
            workshop_session_url.append(link)
        except:
            continue
    
    else:
        abstract_titles = [item.text for item in soup.find_all('a') 
            if re.search('AbstractDetails', str(item.get('href')))]

        # map back to session id and original link for later joining
        session_ids = [soup.find_all('tr')[2].find('b').text] * len(abstract_links)
        original_link = [link] * len(abstract_links)

        # create list of 
        session_url = session_url + original_link
        abstract_url = abstract_url + abstract_links
        abstract_title = abstract_title + abstract_titles
        session_no_v2 = session_no_v2 + session_ids

    

In [82]:
len(session_url) == len(abstract_url) == len(abstract_title) == len(session_no_v2)

True

In [83]:
len(session_url)

3570

In [84]:
len(workshop_abstract) == len(workshop_session_no) == len(workshop_session_url)

True

In [85]:
len(workshop_abstract)

301

In [86]:
df_abstracts = pd.DataFrame({'Session_URL': session_url,
                            'Abstract_URL' : abstract_url,
                            'Abstract_Title': abstract_title,
                            'Session_No_v2': session_no_v2})

In [87]:
df_abstracts['Session_ID'] = df_abstracts['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))
df_abstracts['Abstract_URL'] = df_abstracts['Abstract_URL'].apply(lambda x: url + x)

In [88]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID
0,Modeling Means and Variances Using Mixed-Effec...,https://ww2.amstat.org/meetings/jsm/2016/onlin...,1,https://ww2.amstat.org/meetings/jsm/2016/onlin...,213128


In [89]:
df_workshops = pd.DataFrame({'Session_No': workshop_session_no,
                            'Abstract_Text': workshop_abstract,
                            'Session_URL': workshop_session_url})


In [90]:
df_workshops['Session_ID']= df_workshops['Session_URL'].apply(lambda x:re.sub(r'https\S+((\d){6})', r'\1', x))
df_workshops = df_workshops[df_workshops['Abstract_Text'].str.contains('Organizer\(s\)|Chair\(s\)') == False]

In [105]:
df_workshops.head(3)

,Session_No,Session_URL,Session_ID,Abstract_Text
0,213714,https://ww2.amstat.org/meetings/jsm/2016/onlin...,213714,"Chair(s): Erin Tanenbaum, NORC at the Univers..."
1,213462,https://ww2.amstat.org/meetings/jsm/2016/onlin...,213462,"Organizer(s): Leah J. Welty, Northwestern Univ..."
2,CE_02C,https://ww2.amstat.org/meetings/jsm/2016/onlin...,212647,Do you get questions about your data after you...


In [106]:
df_abstracts.drop_duplicates(subset = 'Abstract_URL', inplace = True)
df_abstracts.reset_index(drop = True, inplace = True)

In [108]:
df_abstracts.to_excel(r'{}\2016Abstracts.xlsx'.format(dest))
df_workshops.to_excel(r'{}\2016Workshops.xlsx'.format(dest))

# Test extraction of abstract info here

In [111]:
test = df_abstracts['Abstract_URL'].iloc[0]
test

'https://ww2.amstat.org/meetings/jsm/2016/onlineprogram/AbstractDetails.cfm?abstractid=319067'

In [112]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [119]:
re.sub(r'\r|\n|\t', '', soup.find_all('tr')[11].find('p').text)

"Intensive longitudinal data are increasingly encountered in many research areas. For example, ecological momentary assessment (EMA) and/or mobile health (mHealth) methods are often used to study subjective experiences within changing environmental contexts. In these studies, up to 40 observations are usually obtained for each subject over a period of a week or so, allowing one to characterize a subject's mean and variance and specify models for both. In this presentation, we focus on an adolescent smoking study using EMA where interest is on characterizing changes in mood variation. We describe how covariates can influence the mood variances and extend the statistical model by adding a subject-level random effect to the within-subject variance specification. This permits subjects to have influence on the mean, or location, and variability, or (square of the) scale, of their mood responses. Models for both continuous and ordinal outcomes are described and illustrated with examples. The

# Code to extract abstract details from abstract URLs

In [120]:
abs_num = []
abs_keyword = []
abs_type = []
abs_text = []

for link in df_abstracts['Abstract_URL']:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # abstract number
    try:
        number = re.sub(r'\S+abstractid=(\d+)', r'\1', link)
        abs_num.append(number)
    except:
        abs_num.append(None)
    
    # abstract keywords
    try:
        keywords = [item.text for item in soup.find_all('a') if re.search('KeyWordSearch', str(item.get('href')))]
        keywords = ', '.join(keywords)
        abs_keyword.append(keywords)
    except:
        abs_keyword.append(None)
        
    
    # abstract type
    try:
        talk_type = re.sub(r'Type:|\n|\r|\t', '', soup.find_all('tr')[3].text)
        abs_type.append(talk_type)
    except:
        abs_type.append(None)
    
    # abstract text
    try:
        abstract_text = re.sub(r'\r|\n|\t', '', soup.find_all('tr')[11].find('p').text)
        abs_text.append(abstract_text)
    except:
        abs_text.append(None)
    

In [121]:
df_abstracts['Abstract_ID'] = abs_num
df_abstracts['Abstract_Keywords'] = abs_keyword
df_abstracts['Abstract_Type'] = abs_type
df_abstracts['Abstract_Text'] = abs_text

In [122]:
df_abstracts.to_excel(r'{}\2016Abstracts.xlsx'.format(dest))

In [123]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID,Abstract_ID,Abstract_Keywords,Abstract_Type,Abstract_Text
0,Modeling Means and Variances Using Mixed-Effec...,https://ww2.amstat.org/meetings/jsm/2016/onlin...,1,https://ww2.amstat.org/meetings/jsm/2016/onlin...,213128,319067,"heterogeneity, mixed-effects models, variance ...",Roundtables,Intensive longitudinal data are increasingly e...


In [152]:
df_workshops['Abstract_Text'] = df_workshops['Abstract_Text'].apply(lambda x: '' if bool(re.search(r'Chair\(s\):|Organizer\(s\)', x)) else x)



In [154]:
df_details = df_workshops.append(df_abstracts)

In [155]:
df.head(0)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID


In [156]:
df_final = df_details.merge(df[['Session_Title', 'Sponsors', 'Type', 'Session_ID']], on = 'Session_ID', how= 'outer')

In [157]:
df_final['Unique'] = df_final['Session_ID'].map(str) + df_final['Abstract_ID'].map(str)

In [158]:
df_final.drop_duplicates(subset= 'Unique', inplace = True)
df_final.reset_index(drop = True, inplace = True)

In [159]:
df_final['Session_No'] = df_final['Session_No'].apply(lambda x: x if bool(str(x).startswith('CE') or str(x)[0].isdigit()) else "")

In [160]:
df_final['Session_No_Keep'] = df_final['Session_No_v2'].combine_first(df_final['Session_No'])

In [161]:
del df_final['Session_No']
del df_final['Session_No_v2']

In [162]:
df_final.rename(columns = {'Session_No_Keep': 'Session_No'}, inplace=True)

In [164]:
new_dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Final Data'
df_final.to_excel(r'{}\2016_JSM_data.xlsx'.format(new_dest))